In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity
import random
import joblib

In [2]:
df=pd.read_csv("Product.csv")

In [3]:
df.head()

,title,Category,selling_price,product_rating,user_id,links
0,Alisha Solid Women's Cycling Shorts,"""Clothing Women's Clothing Lingerie, Sleep &...",999,5.0,19,https://m.media-amazon.com/images/I/51MmfNZb6T...
1,FabHomeDecor Fabric Double Sofa Bed,"""Furniture Living Room Furniture Sofa Beds &...",32157,3.0,131,/static/img/Image_not_available.png\n
2,AW Bellies,"""Footwear Women's Footwear Ballerinas AW Be...",999,1.6,166,https://m.media-amazon.com/images/I/61DlDX5L0e...
3,Alisha Solid Women's Cycling Shorts,"""Clothing Women's Clothing Lingerie, Sleep &...",699,3.4,12,https://m.media-amazon.com/images/I/51MmfNZb6T...
4,Sicons All Purpose Arnica Dog Shampoo,"""Pet Supplies Grooming Skin & Coat Care Sha...",220,2.9,137,https://m.media-amazon.com/images/I/61OCfq7o8F...


In [4]:
df.value_counts('user_id')

user_id
26     196
140    190
37     189
56     189
93     186
      ... 
19     134
23     133
192    131
104    128
0       96
Length: 201, dtype: int64

In [5]:
df.isna().sum()

title              0
Category           0
selling_price     78
product_rating     0
user_id            0
links              0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.duplicated().sum()

6

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.groupby('user_id').count()['product_rating'].reset_index().describe()

,user_id,product_rating
count,201.000000,201.000000
mean,100.000000,158.547264
std,58.167861,13.747691
min,0.000000,95.000000
25%,50.000000,150.000000
50%,100.000000,159.000000
75%,150.000000,167.000000
max,200.000000,196.000000


In [10]:
cb=df.groupby('user_id').count()['product_rating']>=160
users_active=cb[cb].index

In [11]:
colab_df=df[df['user_id'].isin(users_active)] # All users in users df

In [12]:
colab_df.groupby('title').count()['product_rating'].reset_index().describe()

,product_rating
count,13006.000000
mean,1.276642
std,1.569035
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,58.000000


In [13]:
colab_df.groupby('title').count()['product_rating'].reset_index().describe()

,product_rating
count,13006.000000
mean,1.276642
std,1.569035
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,58.000000


In [14]:
fpr=colab_df.groupby('title').count()['product_rating']>=5
product_famous=fpr[fpr].index

In [15]:
colab_df=colab_df[colab_df['title'].isin(product_famous)]

In [16]:
colab_df.describe()

,product_rating,user_id
count,2181.000000,2181.000000
mean,3.092710,93.794590
std,1.168488,60.916253
min,1.000000,1.000000
25%,2.100000,37.000000
50%,3.100000,90.000000
75%,4.100000,145.000000
max,5.000000,200.000000


In [17]:
colab_df.duplicated().sum()

0

In [18]:
pt=colab_df.pivot_table(index="title",columns="user_id",values="product_rating")

In [19]:
pt.fillna(0,inplace=True)

In [20]:
pt

user_id,1,2,4,5,7,8,10,11,12,14,...,182,183,184,188,190,191,194,197,198,200
title,,,,,,,,,,,,,,,,,,,,,
925 Silver Alloy Silver Bracelet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.6,0.0,0.0,0.0,0.0,0.0
925 Silver Silver Bracelet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.8,0.0,4.3,0.0
999store Medium Paper Sticker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,0.0
ABHIRAAM Men Regular Fit Blue Cotton Blend Trousers,0.0,0.0,3.4,0.0,0.0,3.4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alan Jones Boys Full Sleeve Color Block Hooded Sweatshirt,0.0,0.0,0.0,0.0,0.0,4.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i-KitPit Pouch for Lenovo A369i,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kasemantra Back Cover for Apple iPad Mini,0.0,0.0,0.0,0.0,4.3,0.0,0.0,0.0,0.0,0.0,...,4.2,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
meSleep Abstract Cushions Cover,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
similarity_score=cosine_similarity(pt)

In [22]:
similarity_score[5]

array([0.2040211 , 0.        , 0.        , 0.26128216, 0.23287952,
       1.        , 0.        , 0.2164538 , 0.04600732, 0.31639965,
       0.09036175, 0.        , 0.        , 0.08429349, 0.11614692,
       0.        , 0.        , 0.        , 0.14945924, 0.07965471,
       0.25296409, 0.07211192, 0.21014447, 0.        , 0.11282389,
       0.        , 0.11991359, 0.10722726, 0.        , 0.        ,
       0.27119373, 0.13727404, 0.1378025 , 0.        , 0.04623563,
       0.13645687, 0.        , 0.        , 0.        , 0.13117947,
       0.11858325, 0.05820964, 0.21824817, 0.10979842, 0.        ,
       0.28262711, 0.12248387, 0.        , 0.        , 0.        ,
       0.        , 0.12074147, 0.08403984, 0.22103317, 0.        ,
       0.07029236, 0.1798831 , 0.        , 0.12502273, 0.1301691 ,
       0.01857306, 0.24306885, 0.23371564, 0.13874218, 0.12977644,
       0.19113137, 0.17766404, 0.18106246, 0.14979906, 0.17907817,
       0.        , 0.1221119 , 0.04159265, 0.06431734, 0.11607

In [96]:
def recommend(product_title):
    index=np.where(pt.index==product_title)[0][0]
#     print(index)
    similar_products=sorted(list(enumerate(similarity_score[index])),key= lambda x:x[1],reverse=True)[1:6]
    l=[]
    for i in similar_products:
        a=df[df['title']==pt.index[i[0]]]['links']
        a=a.tolist()[0][:-1]
        l.append(pt.index[i[0]])
    return l

In [97]:
colab_df
# recommend(m)

,title,Category,selling_price,product_rating,user_id,links
49,MASARA Solid Women's Straight Kurta,"""MASARA Solid Women's Straight Kurt",1399,3.7,155,https://m.media-amazon.com/images/I/71TS7epDqI...
57,MASARA Solid Women's Straight Kurta,"""MASARA Solid Women's Straight Kurt",1399,3.5,47,https://m.media-amazon.com/images/I/51FPINpjXv...
58,MASARA Solid Women's Straight Kurta,"""MASARA Solid Women's Straight Kurt",1399,2.6,125,https://m.media-amazon.com/images/I/71TS7epDqI...
62,MASARA Solid Women's Straight Kurta,"""MASARA Solid Women's Straight Kurt",1399,3.0,52,https://m.media-amazon.com/images/I/717fBVuoZG...
66,MASARA Solid Women's Straight Kurta,"""MASARA Solid Women's Straight Kurt",999,1.4,100,https://m.media-amazon.com/images/I/71TS7epDqI...
...,...,...,...,...,...,...
30749,Alan Jones Boys Full Sleeve Color Block Hooded...,Bady and Kids Boys Winter Wear,₹610,4.4,45,https://m.media-amazon.com/images/I/61qQJOFhYc...
30896,Alan Jones Boys Full Sleeve Color Block Hooded...,Bady and Kids Boys Winter Wear,₹564,4.3,190,https://m.media-amazon.com/images/I/61qQJOFhYc...
31221,Alan Jones Boys Full Sleeve Color Block Hooded...,Bady and Kids Boys Winter Wear,₹499,4.2,8,https://m.media-amazon.com/images/I/61qQJOFhYc...
31289,Allen Solly Boys Solid Casual Jacket,Bady and Kids Girls Winter Wear,₹929,4.7,58,https://m.media-amazon.com/images/I/51vwTSzkDj...


In [98]:
m="MASARA Solid Women's Straight Kurta"
recommend(m)

["Ninja Turtle Printed Men's Round Neck T-Shirt",
 "Blenni Women's Solid Casual Shirt",
 'shenthri\xa0Solid Men Waistcoat',
 "CampusMall Printed Men's Round Neck T-Shirt",
 'Hi Look Alloy Necklace']

## Storing Pt dataframe and similarity score

In [62]:
joblib.dump(pt,"Collabarative.sav")

['Collabarative.sav']

In [63]:
joblib.dump(similarity_score,"similarity.sav")

['similarity.sav']